In [1]:
%matplotlib inline 
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# データ
train_data = pd.read_csv("NKI_RS_train_data.csv",index_col=0)
train_label = pd.read_csv("NKI_RS_train_label.csv",index_col = 0)
test_data = pd.read_csv("NKI_RS_test_data.csv", index_col = 1)
train_label = train_label["sex"]

In [3]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state = 0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 訓練用の関数
from train_eval_loop import train_valid_loop
# データをtensor型にかえる関数
from datapreparation import df2tensor

In [4]:
# modelのimport
from model import Net_simple, Net_dp, Net_bn

In [6]:
from train_eval_loop_test import train_valid_loop
num = 0
n_epoch = 100
learning_rate = 1e-3
for train_idx, valid_idx in skf.split(train_data, train_label):
    print(f"This is {num + 1}th fold.100epochごとの結果を以下に出力--------------------")
    model = Net_simple().to(device)
    optimizer = optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9, weight_decay = 5e-3)
    best_score = 0
    trainloader, validloader, valid_data, valid_label = df2tensor(
        train_data.iloc[train_idx], train_label.iloc[train_idx], train_data.iloc[valid_idx], train_label.iloc[valid_idx], batch_size = 64)
    train_acc_list, train_loss_list, valid_acc_list, valid_loss_list, auc_score_list = train_valid_loop(
        trainloader, validloader, valid_data, valid_label, model, n_epoch, optimizer)
    auc_score = auc_score_list[-1]
    if auc_score > best_score:
        best_score = auc_score
        best_model = model
    
    for a,b,c,d,e in zip(
        train_loss_list, train_acc_list, valid_loss_list, valid_acc_list, auc_score_list):
        print(f"train loss:{a:.5f},train acc:{b:.5f}, valid loss:{c:.5f}, valid acc:{d:.5f}, auc_score:{e:.5f}")
    num += 1

This is 1th fold.100epochごとの結果を以下に出力--------------------
train loss:0.69096,train acc:0.60802, valid loss:0.68900, valid acc:0.60494, auc_score:0.50000
train loss:0.66949,train acc:0.60802, valid loss:0.67087, valid acc:0.60494, auc_score:0.50000
This is 2th fold.100epochごとの結果を以下に出力--------------------
train loss:0.71019,train acc:0.39198, valid loss:0.70595, valid acc:0.39506, auc_score:0.50000
train loss:0.66953,train acc:0.60802, valid loss:0.67092, valid acc:0.60494, auc_score:0.50000
This is 3th fold.100epochごとの結果を以下に出力--------------------
train loss:0.68386,train acc:0.60802, valid loss:0.68292, valid acc:0.60494, auc_score:0.50000
train loss:0.66962,train acc:0.60802, valid loss:0.67093, valid acc:0.60494, auc_score:0.50000
This is 4th fold.100epochごとの結果を以下に出力--------------------
train loss:0.68571,train acc:0.60648, valid loss:0.68376, valid acc:0.61111, auc_score:0.50000
train loss:0.67032,train acc:0.60648, valid loss:0.66821, valid acc:0.61111, auc_score:0.50000
This is 5th 

In [7]:
# dropoutを入れたネットワーク
num = 0
n_epoch = 500
for train_idx, valid_idx in skf.split(train_data, train_label):
    print(f"This is {num + 1}th fold.100epochごとの結果を以下に出力--------------------")
    model = Net_dp().to(device)
    optimizer = optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9, weight_decay = 5e-3)
    best_score = 0
    trainloader, validloader, valid_data, valid_label = df2tensor(
        train_data.iloc[train_idx], train_label.iloc[train_idx], train_data.iloc[valid_idx], train_label.iloc[valid_idx], batch_size = 64)
    train_acc_list, train_loss_list, valid_acc_list, valid_loss_list, auc_score_list = train_valid_loop(
        trainloader, validloader, valid_data, valid_label, model, n_epoch, optimizer)
    auc_score = auc_score_list[-1]
    if auc_score > best_score:
        best_score = auc_score
        best_model = model
    
    for a,b,c,d,e in zip(
        train_loss_list, train_acc_list,
        valid_loss_list, valid_acc_list, auc_score_list):
        print(f"train loss:{a:.5f},train acc:{b:.5f}, valid loss:{c:.5f}, valid acc:{d:.5f}, auc_score:{e:.5f}")
    num += 1

This is 1th fold.100epochごとの結果を以下に出力--------------------
train loss:0.69564,train acc:0.40432, valid loss:0.69287, valid acc:0.60494, auc_score:0.50000
train loss:0.66950,train acc:0.60802, valid loss:0.67092, valid acc:0.60494, auc_score:0.50000
train loss:0.66979,train acc:0.60802, valid loss:0.67096, valid acc:0.60494, auc_score:0.50000
train loss:0.66950,train acc:0.60802, valid loss:0.67096, valid acc:0.60494, auc_score:0.50000
train loss:0.66969,train acc:0.60802, valid loss:0.67096, valid acc:0.60494, auc_score:0.50000
train loss:0.66977,train acc:0.60802, valid loss:0.67096, valid acc:0.60494, auc_score:0.50000
This is 2th fold.100epochごとの結果を以下に出力--------------------
train loss:0.68892,train acc:0.60802, valid loss:0.68750, valid acc:0.60494, auc_score:0.50000
train loss:0.66969,train acc:0.60802, valid loss:0.67095, valid acc:0.60494, auc_score:0.50000
train loss:0.66943,train acc:0.60802, valid loss:0.67096, valid acc:0.60494, auc_score:0.50000
train loss:0.66966,train acc:0.

KeyboardInterrupt: 

In [8]:
# batch normalizationをいれたネットワーク
num = 0
n_epoch = 500
for train_idx, valid_idx in skf.split(train_data, train_label):
    print(f"This is {num + 1}th fold.100epochごとの結果を以下に出力--------------------")
    model = Net_bn().to(device)
    optimizer = optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9, weight_decay = 5e-3)
    best_score = 0
    trainloader, validloader, valid_data, valid_label = df2tensor(
        train_data.iloc[train_idx], train_label.iloc[train_idx], train_data.iloc[valid_idx], train_label.iloc[valid_idx], batch_size = 64)
    train_acc_list, train_loss_list, valid_acc_list, valid_loss_list, auc_score_list = train_valid_loop(
        trainloader, validloader, valid_data, valid_label, model, n_epoch, optimizer)
    auc_score = auc_score_list[-1]
    if auc_score > best_score:
        best_score = auc_score
        best_model = model
    
    for a,b,c,d,e in zip(
        train_loss_list, train_acc_list,
        valid_loss_list, valid_acc_list, auc_score_list):
        print(f"train loss:{a:.5f},train acc:{b:.5f}, valid loss:{c:.5f}, valid acc:{d:.5f}, auc_score:{e:.5f}")
    num += 1

This is 1th fold.100epochごとの結果を以下に出力--------------------
train loss:0.66389,train acc:0.61265, valid loss:0.67359, valid acc:0.60494, auc_score:0.50000
train loss:0.31616,train acc:1.00000, valid loss:0.61620, valid acc:0.67284, auc_score:0.51562
train loss:0.31683,train acc:1.00000, valid loss:0.62652, valid acc:0.66667, auc_score:0.61097
train loss:0.31718,train acc:1.00000, valid loss:0.62825, valid acc:0.66667, auc_score:0.64764
train loss:0.39080,train acc:0.92284, valid loss:0.64061, valid acc:0.66049, auc_score:0.61607
train loss:0.32940,train acc:0.98765, valid loss:0.57443, valid acc:0.74074, auc_score:0.64461
This is 2th fold.100epochごとの結果を以下に出力--------------------
train loss:0.66315,train acc:0.61111, valid loss:0.68233, valid acc:0.60494, auc_score:0.50000
train loss:0.31653,train acc:1.00000, valid loss:0.60276, valid acc:0.65432, auc_score:0.61543
train loss:0.31712,train acc:1.00000, valid loss:0.60121, valid acc:0.66667, auc_score:0.70504
train loss:0.31717,train acc:1.